<center>
    <header>
        <h1>Analisi dello Stack TCP-IP</h1>
        <h2>Marco Bondaschi, Mauro Conte</h2>
        <h4>Laboratorio di Telecomunicazioni</h4>
        <h5>Università Degli Studi Di Brescia</h5>
        <h6>22/05/2017</h6>
    </header>
</center>
<hr><hr>

<h2>Descrizione</h2>

Con quest'ultima esperienza abbiamo osservato ed analizzato il funzionamento dello stack protocollare TCP-IP, sperimentando i diversi protocolli che esso utilizza per gestire le più svariate attività, e verificando le caratteristiche dei vari pacchetti che entrano in gioco durante la comunicazione tra due terminali. Grazie a questa attività abbiamo quindi sperimentato le diverse opzioni offerte da un classico analizzatore di protocollo, similmente a quanto fatto durante la precedente esercitazione.


<h2>Strumenti</h2>

Gli strumenti che abbiamo utilizzato per svolgere questa esperienza sono:
<ul>
<li>I nostri due computer Windows, su cui è stato installato l'analizzatore di protocollo <i>Wireshark</i>;</li>
<li>I due PC in dotazione al laboratorio, su cui è presente l'analizzatore di protocollo <i>Fluke Protocol Inspector</i>;</li>
<li>Un cavo coassiale con cui collegare tra loro le schede di rete dei due PC;</li>
<li>Il software <i>Microsoft Excel</i> per l'elaborazione dei dati ottenuti dall'analisi di protocollo.</li>
</ul>


<h2>Svolgimento</h2>
<h3>Configurazione della postazione di lavoro</h3>

Per svolgere la prima parte dell'esperienza abbiamo utilizzato i nostri due PC, su cui abbiamo installato <i>Wireshark</i>. Poichè la rete universitaria impediva di inviare pacchetti ping tra i due terminali, abbiamo optato per l'utilizzo di un <i>hotspot</i> tramite un nostro cellulare, a sua volta collegato alla rete mobile. Una volta collegati i due PC all'hotspot, abbiamo potuto procedere con l'esperienza.


<h3>ARP (Address Resolution Protocol) e DNS (Domain Name Server)</h3>

Per prima cosa abbiamo recuperato da entrambi i PC Windows i relativi indirizzi MAC e IP tramite il DOS attraverso il comando <i>ipconfig /all</i>. Il primo PC aveva come indirizzo MAC F0:7B:CB:1F:E8:13 e come indirizzo IP 192.168.43.147, mentre il secondo aveva come indirizzo MAC 30:59:B7:0D:73:42 e come indirizzo IP 192.168.43.106.
Abbiamo quindi impostato come filtro di cattura di Wireshark <i>eth.addr == F0:7B:CB:1F:E8:13 || eth.addr == 30:59:B7:0D:73:42</i> in modo da limitare la cattura dei pacchetti sono per gli indirizzi MAC dei nostri due PC.
Per testare l'invio e ricezione di pacchetti ARP abbiamo quindi trasmesso dei dati da un terminale all'altro tramite il comando da DOS <i>ping 30:59:B7:0D:73:42</i>. Una volta completata la procedura abbiamo terminato la cattura con Wireshark e abbiamo analizzato il traffico di pacchetti tra i due PC.

<h4>Domande</h4>

<h5>1) Qual è l’impiego del protocollo ARP?</h5>
ARP (<i>Address Resolution Protocol</i>) è un protocollo di rete dello stack TCP-IP che permette di collegare l'indirizzo IP delle macchine (che le identifica univocamente sulla rete) con il rispettivo indirizzo fisico MAC (che indentifica univocamente le schede di rete indipendentemente dalla rete in cui sono connesse).

<h5>2) Come è identificato il protocollo ARP? Come è trasportato?</h5>
Il pacchetto ARP è incapsulato nel payload del pacchetto Ethernet, ed è identificato nel campo EtherType tramite il codice 0x0806. La richiesta dell'indirizzo fisico di un certo IP viene indirizzata in broadcast, mentre la risposta è inviata direttamente all'indirizzo MAC richiedente.

<h5>3) Quale è l’indirizzo MAC che il PC inizia a risolvere per primo? Chi fornisce la risposta?</h5>
Il PC che ha inviato il ping invia per primo un pacchetto ARP in broadcast richiedente l'indirizzo fisico del router, identificato dall'indirizzo IP 192.168.43.1. Il router risponde quindi al primo PC indicandogli il suo indirizzo fisico (nel nostro caso esso è risultato essere 5E:AF:06:6C:19:87). Successivamente il PC mittente invia un altro pacchetto ARP diretto all'indirizzo MAC del router chiedendogli l'indirizzo fisico corrispondente all'IP del PC destinatario, ed il router risponde con un ARP reply contenente il MAC di destinazione.

<h5>4) Qual è la ragione della richiesta DNS? Chi è l’autore della risposta?</h5>
Per svolgere questa richiesta abbiamo inviato con il primo PC un ping ad un indirizzo web arbitrario (<i>ping google.it</i>), ed abbiamo analizzato il traffico di pacchetti con Wireshark in modo del tutto simile a quanto visto in precedenza. In questo caso, la risposta che include l'indirizzo IP del destinatario è contenuta in un pacchetto DNS, inviato al computer mittente dal router (192.168.43.1).

<h5>5) Analizzare e commentare i parametri della risposta DNS.</h5>
Il pacchetto contenente la DNS reply è trasportato da UDP. Tra i campi contenuti nel pacchetto DNS sono presenti la <i>Flag DNS</i>, che nel nostro caso conteneva il codice 0x8180 che identifica una <i>Standard Query Response</i>, ed i campi risposta: <i>Name</i> (google.it), <i>Type</i> (A, che indica un <i>Host Address</i>), <i>Class</i> (IN), <i>Time to live</i> (294), <i>Data length</i> (4), <i>Address</i> (216.58.198.35). Inoltre, nel pacchetto UDP è contenuto il campo <i>Source port</i> contenente il valore 53 (porta DNS standard) ed il campo <i>Destination port</i> contenente il valore 56466.

<h5>6) Nella finestra DOS, digitare il commando “arp –a’’ e commentare.</h5>
Digitando dal DOS il comando <i>arp -a</i> viene visualizzata l'intera tabella ARP immagazzinata nella cache locale del sistema operativo. Dopo questa esperienza, la tabella ARP del PC mittente conteneva l'indirizzo IP e MAC di se stesso, del router e del PC destinatario dei dati. La tabella ARP permette al PC di velocizzare l'indirizzamento di pacchetti ad un terminale con cui ha già comunicato in precedenza, memorizzando automaticamente il collegamento tra IP e MAC in modo da non dover mandare nuovamente la richiesta ARP.


<h3>Connessione TCP (Transport Control Protocol)</h3>

Per analizzare le funzionalità del protocollo TCP, invece, abbiamo avuto bisogno di instaurare una connessione FTP all'indirizzo <i>yonderlabs.com</i>. Per far ciò, dal DOS del nostro PC abbiamo chiamato il comando <i>ftp yonderlabs.com</i>. In risposta abbiamo ottenuto una richiesta di accesso con nome utente e password. Non possedendo delle credenziali valide, abbiamo inserito informazioni arbitrarie e, una volta che il server FTP ha rifiutato la nostra richiesta di accesso, abbiamo terminato la connessione FTP con il comando <i>quit</i>.
Durante questo processo abbiamo monitorato lo scambio di pacchetti attraverso Wireshark, sulla falsa riga di quanto fatto per la prima parte dell'esperienza, ed abbiamo analizzato quanto ottenuto.

<h4>Domande su IP</h4>

<h5>1) Come viene identificato il protocollo IP nella trama 802.3/Ethernet?</h5>
Nel pacchetto Ethernet viene specificato, nel campo EtherType, che tipo di pacchetto viene trasportato nel payload. Nel nostro caso nel payload è contenuto un pacchetto IPv4, che è identificato dal codice 0x0800.

<h5>2) Qual è la lunghezza della trama Ethernet 802.3?</h5>
Nel nostro caso, la lunghezza totale della trama Ethernet risulta di 66 byte.

<h5>3) E del padding della trama Ethernet/802.3?</h5>
Poichè siamo oltre la dimensione minima del pacchetto Ethernet, che è pari a 64 byte, non risulta necessario lo <i>zero-padding</i> del payload, che viene quindi lasciato inalterato.

<h5>4) E dell’header IP?</h5>
L'header IP è lungo 20 byte, e contiene, tra le altre cose, informazioni sulla versione IP, sulla lugnhezza totale del pacchetto IP, sul protocollo che trasporta e sugli indirizzi IP di sorgente e destinatario.

<h5>5) E dell’header TCP?</h5>
L'header TCP è lungo 32 byte; contiene le porte di sorgente e destinazione, il <i>sequence number</i>, l'<i>Acknowledgment number</i>, ed altre informazioni.

<h5>6) Qual é l’indirizzo IP del PC?</h5>
L'indirizzo IP del campo <i>Source Address</i> del pacchetto IP coincide con quanto letto da DOS con il comando <i>ipconfig /all</i> ed è pari a 192.168.43.147.

<h5>7) Come sono identificati i protocolli TCP?</h5>
I protocolli TCP sono identificati dal codice 06 nel campo <i>Protocol</i> dell'header IP.

<h5>8) Qual è l’utilità del campo “Time to live” posto nell’header IP?</h5>
Nel campo TTL dell'header IP è contenuto il numero di <i>hop</i> che il datagramma IP può compiere prima di essere rimosso dalla circolazione nella rete. Il termine <i>hop</i> indica un passaggio del pacchetto attraverso un qualunque router della rete. Ogni router che riceve un pacchetto IP diminuisce quindi di un'unità il valore del campo TTL prima di immettere nuovamente il datagramma sulla rete. Una volta che il valore di TTL giunge a 0, il pacchetto viene scartato è un messaggio ICMP di errore viene inviato al mittente. Questo processo ha la funzione di evitare che un pacchetto resti per un tempo indeterminato sulla rete senza giungere a destinazione (ovvero che si blocchi in un <i>loop</i>), intasando la rete.

<h5>9) Qual è la lunghezza indicata nel campo “length” dell’header IP?</h5>
Nel campo <i>Total length</i> dell'header IP è indicata la lunghezza totale del pacchetto IP, includendo il campo Data. Nel nostro caso, essa è pari a 52 byte.

<h5>10) A cosa serve il campo TOS (Type of Service) dell’header IP?</h5>
???????? (DA FARE...)

<h4>Domande su TCP</h4>

<h5>1) A cosa servono i campi SYN, FIN, PSH?</h5>

Nell'instaurare la connessione con il server FTP, il computer avvia la cosiddetta <i>Three-way handshake</i>, in modo che il server possa aprire la porta necessaria per comunicare con il client. Il primo passo è l'invio di un pacchetto con flag SYN da parte del client verso il server, che a sua volta risponde con un SYN-ACK con un <i>Acknowledgment Number</i> pari all'intero successivo rispetto al <i>Sequence Number</i> del pacchetto SYN inviato dal client. Infine, il client risponde con un ACK con <i>Sequence Number</i> pari all'<i>Acknowledgment Number</i> del SYN-ACK, e un <i>Acknowledgment Number</i> pari all'intero successivo del <i>Sequence Number</i> del SYN-ACK.
Nel momento in cui il client invia la richiesta di chiudere la connessione FTP al server con il comando <i>quit</i>, il server invia al client un pacchetto FIN-ACK, il client risponde con un ACK ed invia a sua volta al server un pacchetto FIN-ACK. Il server chiude definitivamente la comunicazione inviando un ACK al client; questa procedura di chiusura della connessione è chiamata <i>Four-way handshake</i>.
La funzione del flag PSH è doppia: la sorgente imposta la flag per informare il buffer TCP che i dati devono essere inviati immediatamente, senza attendere che altri dati vengano immagazzinati; il destinatario che legge la flag PSH sa che i dati ricevuti devono essere inviati immediatamente al livello applicativo.

<h5>2) Su quale porta viene effettuata la connessione?</h5>
La connessione viene effettuata sulla porta standard di FTP, ovvero la porta 21.

<h5>3) Cosa rappresentano i campi “Sequence Number” e “Window” dell’intestazione? In quale unità sono specificati?</h5>
Il campo <i>Sequence number</i> indica il numero di sequenza del primo byte di dati inviato nel pacchetto. Se il pacchetto è il primo (quindi ha la flag SYN settata), questo campo contiene un numero casuale che diventa il numero di sequenza del primo byte di dati della connessione. Per i pacchetti successivi, il <i>Sequence number</i> corrisponde al <i>Sequence number</i> del pacchetto precedente a cui viene sommato il numero di byte del campo dati del pacchetto precedente.
Il campo <i>Window</i> indica invece la dimensione della <i>Receive Window</i> del mittente, ovvero del numero di byte che esso può ricevere dal destinatario del pacchetto. Entrambi i campi sono quindi espressi in byte.

<h5>4) Durante la fase di trasferimento dei dati, analizzare l’evoluzione dei numeri di sequenza.</h5>
????? (DA FARE...)


<h3>ICMP (Internet Control Message Protocol)</h3>

Per svolgere questa parte dell'esperienza abbiamo utilizzato nuovamente i nostri due PC, connessi nuovamente ad un hotspot mobile. Gli inidirzzi MAC sono rimasti gli stessi della prima parte dell'esperienza, mentre gli indirizzi IP erano pari a 192.168.0.116 per il primo e 192.168.0.191 per il secondo.
Come fatto in precedenza, dal primo PC abbiamo inviato il comando <i>ping 192.168.0.191</i> tramite DOS, ed abbiamo analizzato con Wireshark i pacchetti scambiati durante la comunicazione.

<h4>Domande</h4>

<h5>1) Qual è il protocollo utilizzato per effettuare le domande d’echo?</h5>
Il protocollo utilizzato per effettuare le domande d’echo è l'ICMP (Internet Control Message Protocol).

<h5>2) Come è identificato questo protocollo?</h5>
Esso è identificato dal codice 01 nel campo <i>Protocol</i> dell'header IP.

<h5>3) Cosa si può dire del livello di trasporto?</h5>
La funzione di trasporto è svolta direttamente dal protocollo ICMP, dal momento che esso è trasportato direttamente nel payload del pacchetto IP senza la presenza di protocolli di trasporto normalmente utilizzati da IP per il trasporto di dati, come TCP o UDP. Poichè ICMP non garantisce affidabilità al sottolivello IP, esso è in genere utilizzato solo per inviare messaggi di errore, oppure, come nel nostro caso, per le <i>echo requests</i> e <i>echo replies</i> inviate durante un ping.

<h5>4) Analizzare e commentare gli indirizzi MAC e IP dei pacchetti scambiati.</h5>
Gli indirizzi MAC e IP dei pacchetti ICMP scambiati sono quelli della sorgente e del destinatario locali tra cui è avvenuto il ping, ovvero i nostri due PC.
